In [5]:
from utils import func_to_def, extract_function_calls, safer_eval, chat_completion_openai
from nexus_relational import *
import re
import pprint

In [6]:
functions = [
                get_user_info,
                get_location_info,
                get_food_info,
                get_user_name,
                list_user_ids,
                find_users_by_name,
                find_locations_by_name,
                find_foods_by_name,
                get_user_email,
                get_user_location,
                get_user_favorite_color,
                get_user_favorite_foods,
                get_weather_at_location,
                get_city_for_location,
                get_current_time_for_location,
                get_current_weather_for_location,
                get_food_name,
                get_food_calories,
                get_food_allergic_ingredients,
                get_current_user_id
            ]

In [7]:
functional_context = ""

for f in functions:
    functional_context += func_to_def(f) + "\n"

print(functional_context)


def get_user_info(id: int) -> dict -> dict:
    """Find the user with the given user ID.
    
    Args:
        id: The user's ID.
    
    Returns:
        The user's data."""

def get_location_info(id: int) -> dict -> dict:
    """Find the location with the given location ID.
    
    Args:
        id: The location's ID.
    
    Returns:
        The location's data."""

def get_food_info(food_id: int) -> dict -> dict:
    """Find the food with the given food ID.
    
    Args:
        food_id: The food's ID.
    
    Returns:
        The food's data."""

def get_user_name(user_id: int) -> str -> str:
    """Get the name of the user with the given user ID.
    
    Args:
        user_id: The user's ID.
    
    Returns:
        The user's name."""

def list_user_ids() -> List[str] -> List[str]:
    """List all the user IDs."""

def find_users_by_name(name: str):
    """Find users with the given name.
    
    Args:
        name: The name to search for.
    
    Returns:
        The l

In [8]:
with open("prompts/prompt1.txt") as fin:
    prompt = fin.read()

prompt = prompt.replace("{{FUNCTIONS}}", functional_context)
print(prompt)

You are an AI agent tasked with exploring a function calling environment. Your goal is to analyze the provided functions, understand their potential uses, and propose function calls to resolve any ambiguities or missing information. This process is similar to running unit tests to better understand the function calling environment.

Here is the list of functions you have access to:

<functions>
def get_user_info(id: int) -> dict -> dict:
    """Find the user with the given user ID.
    
    Args:
        id: The user's ID.
    
    Returns:
        The user's data."""

def get_location_info(id: int) -> dict -> dict:
    """Find the location with the given location ID.
    
    Args:
        id: The location's ID.
    
    Returns:
        The location's data."""

def get_food_info(food_id: int) -> dict -> dict:
    """Find the food with the given food ID.
    
    Args:
        food_id: The food's ID.
    
    Returns:
        The food's data."""

def get_user_name(user_id: int) -> str

In [18]:
api_dict = {
    'api_key': '-',
    'api_base': None
}

conversation = []

prefix = "Here are the execution results:\n\n"

conversation.append(
    {
        'role': 'user',
        'content': prompt
    }
)
completion = chat_completion_openai("gpt-4o", conversation, 0.0, 8192, api_dict=api_dict)
print("="*20 + "Competion" + "="*20 + "\n")
print(completion)
raw_calls_str = re.findall(r'<function_list>(.*)</function_list>', completion, flags=re.DOTALL)[0].strip()
calls = extract_function_calls(raw_calls_str)
print("\n" + "="*20 + "Calls" + "="*20 + "\n")
print(calls)
print("\n" + "="*20 + "Call Outputs" + "="*20 + "\n")
context = vars() # you HAVE to do this here!
call_dict = dict((call, safer_eval(call, context=context)) for call in calls)
pprint.pprint(call_dict, sort_dicts=False)

conversation.append(
    {
        'role': 'assistant',
        'content': completion
    }
)

conversation.append(
    {
        'role': 'user',
        'content': prefix + str(call_dict)
    }
)


====================Competion====================

<exploration_summary>
<function_analysis>
- Function name: get_user_info
  - Proposed function call(s): get_user_info(1)
  - Results and observations: Expecting a dictionary with user data.
  - Clarified information: Confirms the return type is a dictionary.
  - Potential use cases: Retrieve detailed information about a specific user.

- Function name: get_location_info
  - Proposed function call(s): get_location_info(1)
  - Results and observations: Expecting a dictionary with location data.
  - Clarified information: Confirms the return type is a dictionary.
  - Potential use cases: Retrieve detailed information about a specific location.

- Function name: get_food_info
  - Proposed function call(s): get_food_info(1)
  - Results and observations: Expecting a dictionary with food data.
  - Clarified information: Confirms the return type is a dictionary.
  - Potential use cases: Retrieve detailed information about a specific food item.

In [19]:
completion = chat_completion_openai("gpt-4o", conversation, 0.0, 8192, api_dict=api_dict)
print("="*20 + "Competion" + "="*20 + "\n")
print(completion)

try:
    raw_calls_str = re.findall(r'<function_list>(.*)</function_list>', completion, flags=re.DOTALL)[0].strip()
    calls = extract_function_calls(raw_calls_str)
    print("\n" + "="*20 + "Calls" + "="*20 + "\n")
    print(calls)
    print("\n" + "="*20 + "Call Outputs" + "="*20 + "\n")
    context = vars() # you HAVE to do this here!
    call_dict = dict((call, safer_eval(call, context=context)) for call in calls)
    pprint.pprint(call_dict, sort_dicts=False)

    conversation.append(
        {
            'role': 'assistant',
            'content': completion
        }
    )
    conversation.append(
        {
            'role': 'user',
            'content': prefix + str(call_dict)
        }
    )
except Exception:
    conversation.append(
        {
            'role': 'assistant',
            'content': completion
        }
    )
    print("No function calls listed, move to next stage.")

====================Competion====================

<exploration_summary>
<function_analysis>
- Function name: get_user_info
  - Proposed function call(s): get_user_info(1)
  - Results and observations: Returned a dictionary with keys 'id', 'name', 'email', 'location', 'favorite_color', and 'favorite_foods'.
  - Clarified information: Confirms the return type is a dictionary with detailed user information.
  - Potential use cases: Retrieve comprehensive user data for profile display or analysis.

- Function name: get_location_info
  - Proposed function call(s): get_location_info(1)
  - Results and observations: Returned a dictionary with keys 'id', 'city', 'current_time', and 'current_weather'.
  - Clarified information: Confirms the return type is a dictionary with detailed location information.
  - Potential use cases: Retrieve location data for travel planning or weather updates.

- Function name: get_food_info
  - Proposed function call(s): get_food_info(1)
  - Results and observati

In [20]:
final_prompt = """Now please update the function descriptions and/or return descriptions to clarify and disambiguate. This will ensure that future calls that do not have access to this conversation will still benefit from this exploration."""

conversation.append(
        {
            'role': 'user',
            'content': final_prompt
        }
    )

completion = chat_completion_openai("gpt-4o", conversation, 0.0, 8192, api_dict=api_dict)
print(completion)



Here are the updated function descriptions and return descriptions based on the exploration results:

```python
def get_user_info(id: int) -> dict:
    """Find the user with the given user ID.
    
    Args:
        id: The user's ID.
    
    Returns:
        A dictionary containing the user's data, including 'id', 'name', 'email', 'location', 'favorite_color', and 'favorite_foods'."""

def get_location_info(id: int) -> dict:
    """Find the location with the given location ID.
    
    Args:
        id: The location's ID.
    
    Returns:
        A dictionary containing the location's data, including 'id', 'city', 'current_time', and 'current_weather'."""

def get_food_info(food_id: int) -> dict:
    """Find the food with the given food ID.
    
    Args:
        food_id: The food's ID.
    
    Returns:
        A dictionary containing the food's data, including 'id', 'name', 'calories', and 'allergic_ingredients'."""

def get_user_name(user_id: int) -> str:
    """Get the name of t